In [ ]:
#idea: first classify between offensive/hateful and not-offensive-nor-hateful (peaceful)
#then classify between offensive and hateful

Step 1: split and label offensive/hateful vs neither

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv('data/train.csv')
df

,id,count,hate_speech,offensive_language,neither,class,tweet
0,12032,6,1,5,0,1,"Im just a young niggah out here chasing ins, t..."
1,9831,3,0,3,0,1,Grant bitches more than a 14 year old white bitch
2,21668,3,0,2,1,1,Stupid lil bitch !! Really think I'm trippen &...
3,16642,3,0,3,0,1,RT @LosBeOnIt: Sex on 1st date dont mean you a...
4,25148,3,0,3,0,1,waaaaay too many niggas comfortable wit dressi...
...,...,...,...,...,...,...,...
19821,25081,3,0,3,0,1,this could be my last breath\r\nI'ma take my t...
19822,683,3,0,3,0,1,#BestSongToHaveSexTo bitches love sosa #bangbang
19823,12583,3,0,3,0,1,"LMAO RT @IGGYAZALEA post away about this ""bitc..."
19824,11217,3,0,3,0,1,I swear to god some hoes look like they take t...


In [11]:
df['class']
df.rename({'class':'tri_class_labels'}, axis='columns', inplace=True)

#re-split data into two groups, if 0/1: hate/offensive group, if 2: neither group
def new_binary_label(old_label):
    #given class label 0,1,2, return 1 if old_label=2 (neither), 0 if old label is either 0 or 1
    if old_label == 2:
        return 1
    return 0



In [12]:
#create new labels (binary) for the dataset and append to df
new_labels = df.apply(lambda row: new_binary_label(row.tri_class_labels), axis=1)
df = df.assign(binary_labels=new_labels.values)
df

,id,count,hate_speech,offensive_language,neither,tri_class_labels,tweet,binary_labels
0,12032,6,1,5,0,1,"Im just a young niggah out here chasing ins, t...",0
1,9831,3,0,3,0,1,Grant bitches more than a 14 year old white bitch,0
2,21668,3,0,2,1,1,Stupid lil bitch !! Really think I'm trippen &...,0
3,16642,3,0,3,0,1,RT @LosBeOnIt: Sex on 1st date dont mean you a...,0
4,25148,3,0,3,0,1,waaaaay too many niggas comfortable wit dressi...,0
...,...,...,...,...,...,...,...,...
19821,25081,3,0,3,0,1,this could be my last breath\r\nI'ma take my t...,0
19822,683,3,0,3,0,1,#BestSongToHaveSexTo bitches love sosa #bangbang,0
19823,12583,3,0,3,0,1,"LMAO RT @IGGYAZALEA post away about this ""bitc...",0
19824,11217,3,0,3,0,1,I swear to god some hoes look like they take t...,0


### Actual Hierarchical classification model (train & predict)

In [62]:
from sklearn.linear_model import LogisticRegression
rand_seed = 42
from sklearn.linear_model import SGDClassifier


def new_binary_label(old_label):
    #given class label 0,1,2, return 1 if old_label=2 (neither), 0 if old label is either 0 or 1
    if old_label == 2:
        return 1
    return 0

def y_train_binary_hsol_vs_neither(tri_labels):
    #helper function, converts tri-classification label to bi-classification (false=H/O, true is neither)
    return tri_labels.between(1.9, 2.1)


def train(X_train, y_train):
    #train step 1 classifier in the hierarchy: [hatespeech or offensive language] vs [neither]
    ytrain_binary_neither = y_train_binary_hsol_vs_neither(y_train)#yields binary labels, 0= either H or O; 1 is neither
    
    lgs_reg_neither = SGDClassifier(random_state=rand_seed)#TEST ONLY
    #lgs_reg_neither = LogisticRegression(random_state=rand_seed)
    lgs_reg_neither.fit(X_train, ytrain_binary_neither)
    
    #train step2 classifier in the hierarchy: [hate speech] vs [offensive language]
    #exclude neither
    y_train_offensive_hate = y_train.loc[ytrain_binary_neither != True]
    X_train_offensive_hate = X_train.loc[ytrain_binary_neither != True] #include only HS or OL rows for 2nd level of hierarchy
    
    lgs_reg_ol_hs = SGDClassifier(random_state=rand_seed)#TEST ONLY
    #lgs_reg_ol_hs = LogisticRegression(random_state=rand_seed)
    #print(y_train_offensive_hate)
    lgs_reg_ol_hs.fit(X_train_offensive_hate, y_train_offensive_hate)
    return lgs_reg_neither, lgs_reg_ol_hs


def predict(X, neither_model, ho_model):
    #first predict [hatespeech or offensive language] vs [neither]
    binary_preds_1 = neither_model.predict(X) #returns False for H/O, True for neither
    y_preds_int = binary_preds_1.astype(int)
    #print(y_preds_int)
    y_preds = y_preds_int * 2 #convert the neither predictions to final label 2
    #then predict [offensive language] vs [hate speech]
    for i in range(len(y_preds)):
        if y_preds[i] == 0:
            y_preds[i] = ho_model.predict(X.iloc[[i]])
    return y_preds


### Test the model with fake data

In [63]:
X_train = df[['id', 'count', 'hate_speech', 'offensive_language', 'neither']]
y_train = df['tri_class_labels']

lgs_reg1, lgs_reg2 = train(X_train, y_train)

In [60]:
import numpy as np

In [64]:
predicted_y = predict(X_train, lgs_reg1, lgs_reg2)
print(predicted_y)

[1 1 1 ... 1 1 1]


In [65]:
print(np.unique(predicted_y))

[0 1 2]


In [70]:
y_real = df['tri_class_labels']
df['predictions'] = predicted_y.tolist()
prediction_correctness = df.apply(lambda row: row.tri_class_labels == row.predictions, axis=1)
print(prediction_correctness)

0         True
1         True
2         True
3         True
4         True
         ...  
19821     True
19822     True
19823     True
19824     True
19825    False
Length: 19826, dtype: bool


In [73]:
print('Count of True values in Column  predictions :', np.count_nonzero(prediction_correctness))
print('percent correctly classified: ', np.count_nonzero(prediction_correctness) / len(prediction_correctness))

Count of True values in Column  predictions : 15297
percent correctly classified:  0.7715625945727832
